In [40]:
using XLSX, JuMP, Dates, DataFrames, Tables;
;

In [41]:
File = "Datos_v8.xlsx"

global cant_trabajadores
global cant_rest_dia 
global cant_rest_bloq

XLSX.openxlsx(File) do xf
    sheet = xf["Trabajador"]
    registros = 0
    for row in XLSX.eachrow(sheet)
        v = row[1]
        if v === nothing || v === "" || ismissing(v)
            break
        end
        registros += 1
    end
    global cant_trabajadores = registros - 1

    registros = 0
    for row in XLSX.eachrow(sheet)
        v = row[8]
        if v === nothing || v === "" || ismissing(v)
            break
        end
        registros += 1
    end
    global cant_rest_dia = registros - 2

    registros = 0
    for row in XLSX.eachrow(sheet)
        v = row[11]
        if v === nothing || v === "" || ismissing(v)
            break
        end
        registros += 1
    end
    global cant_rest_bloq = registros - 2
    
end
println("Cantidad de trabajadores: ", cant_trabajadores)
println("Cantidad de restricciones de dia: ", cant_rest_dia)
println("Cantidad de restricciones de bloque: ", cant_rest_bloq)

;

Cantidad de trabajadores: 18
Cantidad de restricciones de dia: 9
Cantidad de restricciones de bloque: 33


In [42]:
tra_ID = Int.(XLSX.readdata(File, "Trabajador", "A2:A$(1+cant_trabajadores)"))     # identificador inicia en 1
tra_nombre = String.(XLSX.readdata(File, "Trabajador", "B2:B$(1+cant_trabajadores)")) # nombre del trabajador
tra_tipo   = String.(XLSX.readdata(File, "Trabajador", "C2:C$(1+cant_trabajadores)")) # tipo del trabajador (T45, T30, T20, T16)
tra_ante   = Int.(XLSX.readdata(File, "Trabajador", "D2:D$(1+cant_trabajadores)"))    # cantidad de dias trabajados al final de mes anterior
tra_cierre  = Int.(XLSX.readdata(File, "Trabajador", "E2:E$(1+cant_trabajadores)"))   # trabajador habilitado para cierre (1); no habilitado (0)

rest_dia_ID = Int.(XLSX.readdata(File, "Trabajador", "H3:H$(2+cant_rest_dia)"))
rest_dia_dia = Int.(XLSX.readdata(File, "Trabajador", "I3:I$(2+cant_rest_dia)"))

rest_bloq_ID = Int.(XLSX.readdata(File, "Trabajador", "K3:K$(2+cant_rest_bloq)"))
rest_bloq_dia = Int.(XLSX.readdata(File, "Trabajador", "L3:L$(2+cant_rest_bloq)"))
rest_bloq_ini= Int.(XLSX.readdata(File, "Trabajador", "M3:M$(2+cant_rest_bloq)"))
rest_bloq_fin = Int.(XLSX.readdata(File, "Trabajador", "N3:N$(2+cant_rest_bloq)"))

req        = Int.(XLSX.readdata(File, "ReqMin", "C2:I33"))        # requerimiento mínimo de trabajadores para (bloque, día), req[b,d] 
obligatorio = Float64.(XLSX.readdata(File, "ReqMin", "K2:K33"))        # requerimiento mínimo de trabajadores para (bloque, día), req[b,d] 

tipo        = String.(XLSX.readdata(File, "Data", "A2:A5")) 

lectura1  = Int.(XLSX.readdata(File, "Data", "B2:B5"))
h_contr = Dict(tipo[i] => lectura1[i] for i in eachindex(tipo))   # cantidad de horas contratadas, semanal

lectura5   = Int.(XLSX.readdata(File, "Data", "C2:C5"))
d_max = Dict(tipo[i] => lectura5[i] for i in eachindex(tipo))   # cantidad máxima de dias laborables, semanal

max_dias_seguidos = Int(XLSX.readdata(File, "Data", "A11"))   # cantidad máxima de dias consecutivos trabajados
min_dias_dom = Int(XLSX.readdata(File, "Data", "A14"))   # cantidad mínima de domingos libres por mes (T45, T30)

peso_dia = Float64.(XLSX.readdata(File, "Data", "C18:C24"))      
;

In [43]:
# Lee un vectores de tamaño variable
function read_vec_int_by_count(file, sheet; count_ref="A2", col="C", start_row=3)
    n = Int(XLSX.readdata(file, sheet, count_ref))          # cantidad de filas
    n == 0 && return Int[]                                  # por si acaso
    rng = "$(col)$(start_row):$(col)$(start_row + n - 1)"   # rango vertical
    return Int.(vec(XLSX.readdata(file, sheet, rng)))       # Vector{Int}
end 

function read_vec_string_by_count(file, sheet; count_ref="A2", col="C", start_row=3)
    n   = Int(XLSX.readdata(file, sheet, count_ref))
    rng = "$(col)$(start_row):$(col)$(start_row + n - 1)"
    return [String(string(v)) for v in vec(XLSX.readdata(file, sheet, rng))]
end

function read_vec_float_by_count(file, sheet; count_ref="A2", col="C", start_row=3)
    n = Int(XLSX.readdata(file, sheet, count_ref))
    n == 0 && return Float64[]
    rng = "$(col)$(start_row):$(col)$(start_row + n - 1)"
    return Float64.(vec(XLSX.readdata(file, sheet, rng)))
end

function read_vec_time_hhmm_by_count(file, sheet; count_ref="A2", col="F", start_row=3)
    n = Int(XLSX.readdata(file, sheet, count_ref))
    n == 0 && return String[]
    rng = "$(col)$(start_row):$(col)$(start_row + n - 1)"
    raw = XLSX.readdata(file, sheet, rng)
    out = String[]
    for v in vec(raw)
        if v isa Time
            push!(out, Dates.format(v, "HH:MM"))
        elseif v isa AbstractString
            s = strip(v)
            t = tryparse(Time, s)
            if t === nothing
                # por si viene "8.30.00" o "8,30,00"
                s2 = replace(s, '.' => ':', ',' => ':')
                t2 = tryparse(Time, s2)
                push!(out, t2 === nothing ? "" : Dates.format(t2, "HH:MM"))
            else
                push!(out, Dates.format(t, "HH:MM"))
            end
        elseif v isa Real
            # Excel puede guardar horas como fracción del día
            secs = mod(round(Int, v * 24 * 3600), 24*3600)
            push!(out, Dates.format(Time(Dates.Second(secs)), "HH:MM"))
        else
            push!(out, "")
        end
    end
    return out
end

function read_bins_matrix_by_count(file, sheet; count_ref="A2",
                                   start_col="I", end_col="AN", start_row=3)
    n = Int(XLSX.readdata(file, sheet, count_ref))   # número de filas
    last_row = start_row + n - 1                     # = 2 + n si start_row=3
    rng = "$(start_col)$(start_row):$(end_col)$(last_row)"
    return Int.(XLSX.readdata(file, sheet, rng))
end

function read_bins_by_type_count(file;
    sheets = Dict("T45"=>"Jor_T45", "T30"=>"Jor_T30", "T20"=>"Jor_T20", "T16"=>"Jor_T16"),
    count_ref="A2", start_col="I", end_col="AN", start_row=3)

    Dict(tipo => read_bins_matrix_by_count(file, hoja;
                                           count_ref=count_ref,
                                           start_col=start_col,
                                           end_col=end_col,
                                           start_row=start_row)
         for (tipo, hoja) in sheets)
end
;

In [44]:
JT = Dict(
    "T16" => Int(XLSX.readdata(File, "Jor_T16", "A2")),
    "T20" => Int(XLSX.readdata(File, "Jor_T20", "A2")),
    "T30" => Int(XLSX.readdata(File, "Jor_T30", "A2")),
    "T45" => Int(XLSX.readdata(File, "Jor_T45", "A2")), )  

JT_name = Dict(
    "T16" => read_vec_string_by_count(File, "Jor_T16"; count_ref="A2", col="B", start_row=3),
    "T20" => read_vec_string_by_count(File, "Jor_T20"; count_ref="A2", col="B", start_row=3),
    "T30" => read_vec_string_by_count(File, "Jor_T30"; count_ref="A2", col="B", start_row=3),
    "T45" => read_vec_string_by_count(File, "Jor_T45"; count_ref="A2", col="B", start_row=3), )

JT_B_ini = Dict(
    "T16" => read_vec_int_by_count(File, "Jor_T16"; count_ref="A2", col="C", start_row=3),
    "T20" => read_vec_int_by_count(File, "Jor_T20"; count_ref="A2", col="C", start_row=3),
    "T30" => read_vec_int_by_count(File, "Jor_T30"; count_ref="A2", col="C", start_row=3),
    "T45" => read_vec_int_by_count(File, "Jor_T45"; count_ref="A2", col="C", start_row=3), )

JT_B_fin = Dict(
    "T16" => read_vec_int_by_count(File, "Jor_T16"; count_ref="A2", col="D", start_row=3),
    "T20" => read_vec_int_by_count(File, "Jor_T20"; count_ref="A2", col="D", start_row=3),
    "T30" => read_vec_int_by_count(File, "Jor_T30"; count_ref="A2", col="D", start_row=3),
    "T45" => read_vec_int_by_count(File, "Jor_T45"; count_ref="A2", col="D", start_row=3), )

JT_H_trab = Dict(
    "T16" => read_vec_float_by_count(File, "Jor_T16"; count_ref="A2", col="F", start_row=3),
    "T20" => read_vec_float_by_count(File, "Jor_T20"; count_ref="A2", col="F", start_row=3),
    "T30" => read_vec_float_by_count(File, "Jor_T30"; count_ref="A2", col="F", start_row=3),
    "T45" => read_vec_float_by_count(File, "Jor_T45"; count_ref="A2", col="F", start_row=3), )  # JT_H_trab["T45"][26]

JT_H_ini = Dict(
    "T16" => read_vec_time_hhmm_by_count(File, "Jor_T16"; count_ref="A2", col="G", start_row=3),
    "T20" => read_vec_time_hhmm_by_count(File, "Jor_T20"; count_ref="A2", col="G", start_row=3),
    "T30" => read_vec_time_hhmm_by_count(File, "Jor_T30"; count_ref="A2", col="G", start_row=3),
    "T45" => read_vec_time_hhmm_by_count(File, "Jor_T45"; count_ref="A2", col="G", start_row=3), )

JT_H_fin = Dict(
    "T16" => read_vec_time_hhmm_by_count(File, "Jor_T16"; count_ref="A2", col="H", start_row=3),
    "T20" => read_vec_time_hhmm_by_count(File, "Jor_T20"; count_ref="A2", col="H", start_row=3),
    "T30" => read_vec_time_hhmm_by_count(File, "Jor_T30"; count_ref="A2", col="H", start_row=3),
    "T45" => read_vec_time_hhmm_by_count(File, "Jor_T45"; count_ref="A2", col="H", start_row=3), )

cover = read_bins_by_type_count(File)   
; 

In [45]:
trabajadores = 1:cant_trabajadores
dias = 1:7
semanas = 1:4
num_bloques = 32
bloques = 1:num_bloques
;

In [46]:
ModHBK=Model()

#using HiGHS
#set_optimizer(ModHBK, HiGHS.Optimizer) 
#set_optimizer_attribute(ModHBK, "time_limit", 180.0)       # 120 segundos

using HiGHS
model = Model(HiGHS.Optimizer)
set_optimizer_attribute(model, "time_limit", 300.0)


;

### Variables

In [47]:
@variable(ModHBK, y[trabajadores, semanas, dias], Bin) # toma valor 1 si el trabajador t trabaja en semana-dia
@variable(ModHBK, z[t in trabajadores, semanas, dias, j in 1:JT[tra_tipo[t]]], Bin) # toma valor 1 si la jornada j es utilizada por el trabajador t en día d y semana s
@variable(ModHBK, f[semanas, bloques, dias] >=0 )  # variable de exceso para cuantificar para identificar falta de capacidad para semana, dia, bloque
@variable(ModHBK, h[semanas, bloques, dias] >=0 )  # variable de holgura para semana, dia, bloque
@variable(ModHBK, fict[semanas, bloques, dias] >=0 ) ; # variable de ficcion



### Funcion objetivo

In [48]:
@objective(ModHBK, Min, 
    sum(f[s,b,d]*peso_dia[d] + fict[s,b,d]*100*peso_dia[d]  for s in semanas, b in bloques, d in 1:7)  
    + sum(h[s,28,d] for s in semanas, d in 1:7)
    )  ;

In [49]:
# cantidad de días trabajados y descansados semanalmente, para cada trabajador (por defecto quedan acotados los días de descanso)
@constraint(ModHBK, dias_trab_sem_sup[t in trabajadores, s in semanas],
    sum(y[t,s,d] for d in dias) <= d_max[tra_tipo[t]]  )    ;

In [50]:
# algunos tipos de trabajadores solo trabajan Sa y Do
@constraint(ModHBK, jor_FinSem,
    sum(y[t,s,d] for t in trabajadores, s in semanas, d in 1:5 if tra_tipo[t] == "T16") == 0 )  ;

In [51]:
@constraint(ModHBK, trab_dia_si_trabaja[t in trabajadores, s in semanas, d in dias],
    y[t,s,d] == sum(z[t,s,d,j] for j in 1:JT[tra_tipo[t]])  )   ;

In [52]:
# días miercoles y sábado tiene que abrir un T45 en bloque 1
@constraint(ModHBK, open_miercoles_y[s in semanas],
    sum(y[t,s,3] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )
@constraint(ModHBK, open_sabado_y[s in semanas],
    sum(y[t,s,6] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )
@constraint(ModHBK, open_miercoles[s in semanas],
    sum(z[t,s,3,1] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )
@constraint(ModHBK, open_sabado[s in semanas],
    sum(z[t,s,6,1] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )    

# otros días tiene que abrir un T45 en bloque 2, excepto domingos
@constraint(ModHBK, open_otros_y[s in semanas, d in dias; d != 3 && d != 6 && d != 7],
    sum(y[t,s,d] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )
@constraint(ModHBK, open_otros[s in semanas, d in dias; d != 3 && d != 6 && d != 7],
    sum(z[t,s,d,2] for t in trabajadores if tra_tipo[t] == "T45") >= 1 )   ;

In [53]:
# Domingo apertura a las 08:30 (bloque 2), cualquier trabajador
@constraint(ModHBK, open_domingos[s in semanas],
    sum(z[t,s,7,j]*cover[tra_tipo[t]][j,2] for t in trabajadores, j in 1:JT[tra_tipo[t]]) >= 1 )   ; 

In [54]:
# los T45 y T30 deben tener dias domingo libres al mes (como minimo). Modelada como cantidad de domingos trabajados
trabajadores_T30_T45 = [t for t in trabajadores if tra_tipo[t] == "T30" || tra_tipo[t] == "T45"]
@constraint(ModHBK, dom_libres[t in trabajadores_T30_T45],
    sum(y[t,s,7] for s in semanas) <= 4 - min_dias_dom )   ;

In [55]:
# cantidad de dias consecutivos de trabajo limitado a 6 dias (version input)
if false
    dias_totales = [(s,d) for s in semanas for d in dias]  # 28 días del  horizonte
    ventanas_7dias = [dias_totales[i:i+max_dias_seguidos] for i in 2:(length(dias_totales)- max_dias_seguidos)]
    for t in trabajadores
        v = 7 - tra_ante[t]  # cantidad máxima de días consecutivos adicionales permitidos en la primera ventana
        # Primera ventana: tamaño v
        primera_ventana = dias_totales[1:v]
        @constraint(ModHBK,
           sum(y[t,s,d] for (s,d) in primera_ventana) <= v-1 )
        # Resto de ventanas normales (de 7 días)
        for ventana in ventanas_7dias
            @constraint(ModHBK,
                sum(y[t,s,d] for (s,d) in ventana) <= max_dias_seguidos )
        end
    end  
end ;

In [56]:
# horario robusto rotativo
if true
    dias_totales = [(s,d) for s in semanas for d in dias]
    L = length(dias_totales)
    # ventanas cíclicas de 7 días con wrap-around
    ventanas = [ [ dias_totales[mod1(i + k, L)] for k in 0:max_dias_seguidos ]  for i in 1:L ]
    # Sólo T20/T30/T45
    trabajadores_T20_T30_T45 = [t for t in trabajadores if tra_tipo[t] in ("T20","T30","T45")]
    # Máx 6 días trabajados en toda ventana de 7 con wrap-around
    for t in trabajadores_T20_T30_T45
        for i in 1:L
            @constraint(ModHBK, sum(y[t, s, d] for (s,d) in ventanas[i]) <= max_dias_seguidos)
        end
    end 
end ;

In [57]:
# cantidad de horas trabajadas cada semana, para cada trabajador (los dias trabajados -> horas colacion)
@constraint(ModHBK, hr_trab_sem[t in trabajadores, s in semanas],
    sum(JT_H_trab[tra_tipo[t]][j]*z[t,s,d,j] for d in dias, j in 1:JT[tra_tipo[t]]) <= h_contr[tra_tipo[t]] + sum(y[t,s,d]  for d in dias))  ;

In [58]:
# se cubre(soft) el requerimiento mínimo de trabajadores para cada dia-bloque. Se cuantifica el faltante "f", que es minimizado
@constraint(ModHBK, req_minimo[s in semanas, b in bloques, d in dias],
    sum(cover[tra_tipo[t]][j,b]*z[t,s,d,j] for t in trabajadores, j in 1:JT[tra_tipo[t]])  == req[b,d] + h[s,b,d] - f[s,b,d] )  ;

In [59]:
# dias de entrega de mercaderias (miercoles y sabado), bloque 2 debe cubrir el minimo obligatoriamente
# tambien se debe cubrir el minimo obligatoriamente al final de la jornada de trabajo (bloque 31)
#@constraint(ModHBK, req_minimo_obligatorio,
#    sum(f[s,31,d] for s in semanas, d in dias) 
#      +  sum(f[s,2,3] + f[s,2,6] for s in semanas)  
     # +  sum(f[s,3,3] for s in semanas)  # miercoles bloque 3
     # +  sum(f[s,3,6] for s in semanas)  # sabado bloque 3
     # +  sum(f[s,3,d] for s in semanas, d in dias)
#              == 0)  ; 

#@constraint(ModHBK, req_minimo_obligatorio,
#   sum(f[s,b,d]*obligatorio[b] - fict[s,b,d]  for s in semanas, d in dias, b in bloques)  
#         == 0)  ; 

@constraint(ModHBK, req_minimo_obligatorio[s in semanas, d in dias, b in bloques],
   f[s,b,d]*obligatorio[b] - fict[s,b,d]  == 0)  ; 

In [60]:
# El Bloque 1 debe respetar la cantidad de 0 y 1 trabajador dependiendo del día (no debe sobrar trabajador)
@constraint(ModHBK, req_inicio_obligatorio,
    sum(h[s,1,d] for s in semanas, d in dias)   == 0)  ;

In [61]:
# Al cierre de cada dia debe haber 2 trabajadores habilitados (al menos)
@constraint(ModHBK, req_minimo_cierre[s in semanas, d in dias],
    sum( z[t,s,d,j]*tra_cierre[t]*cover[tra_tipo[t]][j,31] for t in trabajadores, j in 1:JT[tra_tipo[t]]) >= 1 ) ; ## modif testig

In [62]:
# Algunos trabajadores no pueden laborar algunos dias
@constraint(ModHBK, No_job_dia,
    sum( y[rest_dia_ID[x],s,rest_dia_dia[x]] for x in 1:cant_rest_dia, s in semanas) == 0 )  ;

In [63]:
# Algunos trabajadores no pueden laborar algunos bloques
con_rest_bloq = Dict{Tuple{Int,Int,Int,Int}, JuMP.ConstraintRef}()

for i in 1:cant_rest_bloq
    t = rest_bloq_ID[i]
    d = rest_bloq_dia[i]
    ini = rest_bloq_ini[i] 
    fin = rest_bloq_fin[i] 
    
    for j in 1:JT[tra_tipo[t]]
        for b in bloques
            if cover[tra_tipo[t]][j,b] == 1 && ( b < ini || b > fin )
                con = @constraint(ModHBK, sum(z[t,s,d,j] for s in semanas) == 0)
                set_name(con, "rest_bloq[$t,$d,$b,$j]")   
                con_rest_bloq[(t,d,b,j)] = con
            end
        end
    end
end ;

#println("Se crearon ", length(con_rest_bloq), " restricciones rest_bloq:")
#for (key, con) in sort!(collect(con_rest_bloq); by=first)
#    println(key, " : ", con)   # imprime la forma simbólica
#end


using JuMP, HiGHS
# Si el modelo ya existe, adjunta el solver; si no, créalo con solver
if @isdefined ModHBK
    set_optimizer(ModHBK, HiGHS.Optimizer)   # <-- clave
else
    ModHBK = Model(HiGHS.Optimizer)
end

set_optimizer_attribute(ModHBK, "time_limit", 300.0)


In [64]:
optimize!(ModHBK)
println("\nStatus = ",termination_status(ModHBK), "   F.Obj = ", objective_value(ModHBK))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 4932 rows; 11228 cols; 151912 nonzeros; 8540 integer variables (8540 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 2e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 4e+01]
Presolving model
1813 rows, 7736 cols, 97300 nonzeros  0s
1685 rows, 7048 cols, 77868 nonzeros  0s

Solving MIP model with:
   1685 rows
   7048 cols (5368 binary, 0 integer, 0 implied int., 1680 continuous, 0 domain fixed)
   77868 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |

In [65]:
"""
construir_cartas_desde_z(
    z, trabajadores, semanas, dias;
    tra_tipo,              # Dict o Vector: trabajador -> "T16"/"T20"/"T30"/"T45"
    JT_H_ini, JT_H_fin,    # Dict{String,Vector{String}} con "HH:MM"
    JT = nothing           # (opcional) Dict{String,Int} con cantidad de jornadas por tipo
)

Genera Dict{eltype(semanas),DataFrame} con una carta por semana.
Cada celda contiene "HH:MM - HH:MM" (o vacío si no hay jornada asignada).
Soporta que z se haya creado con filtro `j <= JT[tra_tipo[t]]`.
"""
function construir_cartas_desde_z(z, trabajadores, semanas, dias;
                                  tra_tipo, JT_H_ini, JT_H_fin,
                                  tra_nombre, JT=nothing)

    # Normaliza a Vector{String} aunque venga Matrix{String}
    nombres = String.(tra_nombre isa AbstractVector ? tra_nombre : vec(tra_nombre))

    JT_local = isnothing(JT) ? Dict(τ => length(JT_H_ini[τ]) for τ in keys(JT_H_ini)) : JT
    @assert length(nombres) == length(trabajadores) "El largo de tra_nombre debe calzar con trabajadores."

    cartas = Dict{eltype(semanas), DataFrame}()

    for s in semanas
        df = DataFrame(Trabajador = nombres)
        for d in dias
            col = Vector{String}(undef, length(trabajadores))
            for (i, t) in enumerate(trabajadores)
                τ  = tra_tipo[t]
                nJ = JT_local[τ]
                rangos = String[]
                for j in 1:nJ
                    if value(z[t, s, d, j]) > 0.5
                        push!(rangos, string(JT_H_ini[τ][j], " - ", JT_H_fin[τ][j]))
                    end
                end
                col[i] = isempty(rangos) ? "" : join(rangos, ", ")
            end
            df[!, Symbol(string(d))] = col
        end
        cartas[s] = df
    end

    return cartas
end ;

In [66]:
cartas = construir_cartas_desde_z(z, trabajadores, semanas, dias;
                                  tra_tipo=tra_tipo, JT_H_ini=JT_H_ini, JT_H_fin=JT_H_fin,
                                  tra_nombre=tra_nombre) ;

In [67]:
cartas[1]

Row,Trabajador,1,2,3,4,5,6,7
,String,String,String,String,String,String,String,String
1,Jocelyn,11:00 - 21:00,08:30 - 18:30,08:00 - 18:00,11:00 - 21:00,08:30 - 18:30,,
2,Ignacio,08:30 - 18:30,11:00 - 21:00,,08:30 - 18:30,11:00 - 21:00,08:00 - 18:00,
3,Ingrid,,13:00 - 20:00,11:00 - 18:00,13:00 - 20:00,15:00 - 22:00,,14:30 - 21:30
4,Agustin,,16:30 - 23:30,16:30 - 23:30,14:30 - 21:30,16:30 - 23:30,,16:30 - 23:30
5,Brian,16:30 - 23:30,16:30 - 23:30,16:30 - 23:30,,14:30 - 21:30,16:30 - 23:30,
6,Franco,09:00 - 16:00,12:00 - 19:00,14:30 - 21:30,,10:00 - 17:00,16:30 - 23:30,
7,Michelle,16:30 - 23:30,,,16:30 - 23:30,16:30 - 23:30,16:30 - 23:30,16:30 - 23:30
8,Sebastian,,10:00 - 17:00,09:00 - 16:00,09:00 - 16:00,09:00 - 16:00,09:00 - 16:00,
9,Moises,12:00 - 19:00,16:30 - 23:30,16:30 - 23:30,16:30 - 23:30,,,10:00 - 17:00


In [68]:
function construir_tablas_f(f, semanas, dias, bloques; empty_for_zero::Bool=true)
    # sanity: verificar que hay solución
    mdl = owner_model(f[first(semanas), first(bloques), first(dias)])
    @assert result_count(mdl) > 0 "Primero corre optimize! para obtener valores de f."

    tablas = Dict{eltype(semanas), DataFrame}()

    for s in semanas
        df = DataFrame(Bloque = collect(bloques))  # filas = bloques
        for d in dias
            vals = [Int(round(value(f[s, b, d]))) for b in bloques]
            if empty_for_zero
                df[!, Symbol(string(d))] = [v == 0 ? "" : string(v) for v in vals]
            else
                df[!, Symbol(string(d))] = vals
            end
        end
        tablas[s] = df
    end

    return tablas
end ;

In [69]:
tablas_f = construir_tablas_f(f, semanas, dias, bloques) ;

In [70]:
df_sin_bloque = select(tablas_f[1], Not(:Bloque))
show(df_sin_bloque; allrows=true, allcols=true, eltypes=false)
println()

32×7 DataFrame
 Row │ 1  2  3  4  5  6  7 
─────┼─────────────────────
   1 │
   2 │
   3 │
   4 │
   5 │
   6 │
   7 │
   8 │
   9 │
  10 │
  11 │
  12 │
  13 │
  14 │
  15 │
  16 │
  17 │
  18 │ 1
  19 │ 1
  20 │
  21 │
  22 │
  23 │
  24 │
  25 │
  26 │
  27 │
  28 │
  29 │
  30 │
  31 │
  32 │


In [71]:
# Muestra si hay mas trabajadores en un bloque >req mínimo (es para mostrar cuando se supera el requerimiento minimo)
if false
    M = [value(h[1,b,d]) for b in bloques, d in dias]   # |bloques| x |dias|
    mask = M .>= 0.5
    
    # índices (i,j) donde pasa el umbral:
    idxs = findall(mask)
    for I in idxs
        b = collect(bloques)[I[1]]
        d = collect(dias)[I[2]]
        println("b=$(b), d=$(d): ", round(M[I], digits=3))
    end
end

In [72]:
function construir_tablas_carga_z(z, trabajadores, semanas, dias, bloques;
                                  tra_tipo, Bbins, JT=nothing, thr=0.5,
                                  empty_for_zero=true, include_bloque=false)

    JT_local = isnothing(JT) ? Dict(τ => size(Bbins[τ], 1) for τ in keys(Bbins)) : JT
    mdl = owner_model(z[first(trabajadores), first(semanas), first(dias), 1])
    @assert result_count(mdl) > 0 "Primero corre optimize! para obtener valores de z."
    max_b = maximum(bloques)
    @assert all(size(Bbins[τ], 2) >= max_b for τ in keys(Bbins)) "Algún Bbins[τ] no cubre todos los bloques."

    tablas = Dict{eltype(semanas), DataFrame}()

    for s in semanas
        df = include_bloque ? DataFrame(Bloque = collect(bloques)) : DataFrame()

        for d in dias
            cont = Vector{Int}(undef, length(bloques))
            for (ib, b) in enumerate(bloques)
                c = 0
                for t in trabajadores
                    τ  = tra_tipo[t]
                    nJ = JT_local[τ]
                    @inbounds for j in 1:nJ
                        if value(z[t, s, d, j]) > thr
                            c += Bbins[τ][j, b]
                        end
                    end
                end
                cont[ib] = c
            end

            df[!, Symbol(string(d))] = empty_for_zero ? [v==0 ? "" : string(v) for v in cont] : cont
        end

        tablas[s] = df
    end

    return tablas
end;

In [73]:
tablas_carga = construir_tablas_carga_z(
    z, trabajadores, semanas, dias, bloques;
    tra_tipo=tra_tipo, Bbins=cover, JT=JT, thr=0.5,
    empty_for_zero=true, include_bloque=false
)

# Imprimir una semana
show(tablas_carga[1]; allrows=true, allcols=true, eltypes=false)
println()

32×7 DataFrame
 Row │ 1  2   3   4   5   6   7  
─────┼───────────────────────────
   1 │        1           1
   2 │ 1  1   2   1   1   2   1
   3 │ 2  2   3   2   2   3   2
   4 │ 2  2   3   2   2   3   2
   5 │ 3  3   3   3   3   3   3
   6 │ 3  3   3   3   3   3   3
   7 │ 4  4   4   4   4   4   4
   8 │ 4  4   4   4   4   4   4
   9 │ 5  5   5   5   5   5   5
  10 │ 5  5   5   5   5   5   5
  11 │ 6  6   6   6   6   6   6
  12 │ 6  6   6   6   6   6   6
  13 │ 6  6   6   6   6   6   6
  14 │ 7  7   8   7   7   7   7
  15 │ 7  7   8   7   8   8   7
  16 │ 7  7   8   7   8   8   7
  17 │ 7  7   7   7   7   7   7
  18 │ 9  10  10  10  10  10  10
  19 │ 8  9   10  9   9   9   9
  20 │ 9  9   10  9   9   9   9
  21 │ 9  9   8   9   9   8   8
  22 │ 8  8   8   8   8   8   8
  23 │ 7  7   7   7   7   7   7
  24 │ 7  7   7   7   7   7   7
  25 │ 6  6   6   6   6   6   6
  26 │ 6  6   6   6   6   6   6
  27 │ 5  5   6   5   5   5   5
  28 │ 4  4   4   4   4   4   4
  29 │ 3  3   3   3   3 

In [74]:
# Convierte un DataFrame en NamedTuple de columnas (lo que XLSX.writetable espera)
function df_a_namedtuple(df::DataFrame)
    syms = Symbol.(names(df))                          # (:Trabajador, :Lunes, ...)
    cols = Tuple(df[!, n] for n in names(df))          # Tuple{Vector,...} mismas longitudes
    return NamedTuple{Tuple(syms)}(cols)
end

function guardar_cartas(cartas::Dict, ruta::AbstractString)
    # Ordena las claves (semanas) para que queden como Semana1, Semana2, ...
    semanas = sort(collect(keys(cartas)))

    # Prepara "parches" hoja=>tabla
    hojas = Vector{Pair{String,NamedTuple}}(undef, length(semanas))
    for (i, s) in enumerate(semanas)
        nombre_hoja = "Semana$(i)"
        df = cartas[s]
        hojas[i] = nombre_hoja => df_a_namedtuple(df)
    end

    # Escribe todas las hojas en una
    XLSX.writetable(ruta, hojas...; overwrite=true)

    return ruta
end

# Uso:
guardar_cartas(cartas, "Carta_output.xlsx")

"Carta_output.xlsx"

In [75]:
# Convierte DataFrame -> NamedTuple de columnas (formato que espera XLSX.writetable)
df_a_namedtuple(df::DataFrame) = NamedTuple{Tuple(Symbol.(names(df)))}(Tuple(df[!, n] for n in names(df)))

"""
guardar_faltantes(tablas_f; ruta="faltantes.xlsx")

Escribe hasta 4 tablas en hojas separadas "Semana1"..."Semana4".
- Soporta Vector{DataFrame} y Dict{<:Integer,DataFrame}.
- Si hay menos de 4, escribe las disponibles.
"""
# Caso 1: Vector de DataFrames
function guardar_faltantes(tablas_f::AbstractVector{<:DataFrame}; ruta::AbstractString="faltantes.xlsx")
    n = min(4, length(tablas_f))
    hojas = [ "Semana$(i)" => df_a_namedtuple(tablas_f[i]) for i in 1:n ]
    XLSX.writetable(ruta, hojas...; overwrite=true)
    return ruta
end

# Caso 2: Dict con claves enteras (p.ej., Dict{Int,DataFrame})
function guardar_faltantes(tablas_f::Dict{<:Integer,<:DataFrame}; ruta::AbstractString="faltantes.xlsx")
    ks = sort(collect(keys(tablas_f)))           # ordena por clave: 1,2,3,4,...
    n  = min(4, length(ks))
    hojas = [ "Semana$(i)" => df_a_namedtuple(tablas_f[ks[i]]) for i in 1:n ]
    XLSX.writetable(ruta, hojas...; overwrite=true)
    return ruta
end

# Uso con tu Dict:
guardar_faltantes(tablas_f; ruta="faltantes.xlsx")

"faltantes.xlsx"